Using publicly available data at data.gov.sg please create a model pipeline to forecast (upcoming 3 hours) the traffic flow at the specified location (latitude: 1.357098686 longitude: 103.902042), for a specified time of day. The solution must have the following components:

1. Estimation of historical traffic flow, using image data sets available here (https://data.gov.sg/dataset/traffic-images )

2. Use the pipeline from (1) and weather data (https://data.gov.sg/dataset/realtime-weather-readings), to forecast the traffic flow at the specified location at a specified time of day

Notes: You may use any additional data sources / APIs to build your models 

# I am running this code on Google Colab. 

In [1]:
!pip install cvlib
!pip install pystan==2.19.1.1
!pip install prophet
!pip install opencv-python==4.1.2.30
!pip install openpyxl

     |████████████████████████████████| 13.1 MB 4.9 MB/s 
  Created wheel for cvlib: filename=cvlib-0.2.7-py3-none-any.whl size=10046385 sha256=27e9a2e3b4cb3315c408606c14b1c78c3c9bafc49e06275c4bf01fb7342d6c22
  Stored in directory: /root/.cache/pip/wheels/8e/d7/31/bc643bd3a8b11a7368b1ab1d8a6299b33b462ed0b0683ddc5a
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=2b31946b6a079ee81b8b6e366643879acff525f09be6a6c56c4aa8e21d1c5882
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built cvlib progressbar
     |████████████████████████████████| 65 kB 2.1 MB/s 
     |████████████████████████████████| 49 kB 4.0 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=6639353 sha256=2d54072e85a01c5726d3533dae3723922e3f7cc6591aa267e3c3f9ac247b845d
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9

In [2]:
# Import Libraries
import requests, json, joblib
import sys
import os
import pandas as pd, numpy as np
from datetime import datetime, timedelta
import pytz
import urllib
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from prophet import Prophet

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/Interviews/Siemens/data/')
print("All the system paths:")
sys.path

All the system paths:


['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/gdrive/My Drive/Colab Notebooks/Interviews/Siemens/data/']

In [5]:
os.chdir('/content/gdrive/My Drive/Colab Notebooks/Interviews/Siemens/data/')
print("Current working directory:")
os.getcwd()

Current working directory:


'/content/gdrive/My Drive/Colab Notebooks/Interviews/Siemens/data'

## Define `cv2plt` for image display

In [ ]:
def cv2plt(img):
    plt.figure(figsize=(8,8))        # To change the size of figure
    plt.axis('off')
    if np.size(img.shape) == 3:
        plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img,cmap='gray',vmin=0,vmax=255)  
    plt.show()

# Using YOLO v3

In [ ]:
lbl_file        = 'yolov3.txt'
classes         = open(lbl_file).read().strip().split("\n")

                                                # Read in the deep learning net
yoloconfig      = 'yolov3.cfg'
yoloweights     = 'yolov3.weights'
net             = cv2.dnn.readNet(yoloweights,yoloconfig)

In [ ]:
def getOutputLayers(net):
    layers = net.getLayerNames()
    outLayers = [layers[i[0]-1] for i in net.getUnconnectedOutLayers()]

    return outLayers


def yoloV3Detect(img,scFactor=1/255,nrMean=(0,0,0),RBSwap=True,scoreThres=0.5,nmsThres=0.4):
    blob = cv2.dnn.blobFromImage(image=img,
                                 scalefactor=scFactor,
                                 size=(416,416),
                                 mean=nrMean,
                                 swapRB=RBSwap,
                                 crop=False)
  
    imgHeight = img.shape[0]
    imgWidth = img.shape[1]

    net.setInput(blob)
    outLyrs = getOutputLayers(net)
    preds = net.forward(outLyrs)

    classId = []
    confidences = []
    boxes = []
    fboxes=[]
    fclasses=[]

    for scale in preds:
      for pred in scale:
        scores = pred[5:]
        clss = np.argmax(scores)
        confidence = scores[clss]

        if confidence > 0.5:
          xc = int(pred[0]*imgWidth)
          yc = int(pred[1]*imgHeight)
          w = int(pred[2]*imgWidth)
          h = int(pred[3]*imgHeight)
          x = xc - w/2
          y = yc - h/2

          classId.append(clss)
          confidences.append(float(confidence))
          boxes.append([x, y, w, h])
      
    selected = cv2.dnn.NMSBoxes(bboxes=boxes,
                                scores=confidences,
                                score_threshold=scoreThres,
                                nms_threshold=nmsThres)
  
  # return empty list 
    if(len(selected) < 1):
      return [fboxes, fclasses]
  
    for j in selected[:,0]:
      fboxes.append(boxes[j])
      fclasses.append(classId[j])

    return [fboxes, fclasses]

In [ ]:
def pltDetect(img, fboxes, fclasses, classes):
    colorset = np.random.uniform(0, 
                                 255,
                                 size=(len(classes), 3))
    txtlbl = ""
  
    for count, fbox in enumerate(fboxes):
      x = int(fbox[0])
      y = int(fbox[1])
      w = int(fbox[2])
      h = int(fbox[3])

      color  = colorset[fclasses[count]]
      txtlbl = str(classes[fclasses[count]])

      cv2.rectangle(img,
                    (x,y),
                    (x+w, y+h),
                    color,
                    2)
    
      cv2.putText(img,
                  txtlbl,
                  (x,y-5),
                  cv2.FONT_HERSHEY_SIMPLEX,
                  0.5,
                  color,
                  1,
                  cv2.LINE_AA)
      
      #cv2plt(img)

      # return number of cars detected
      return (txtlbl.count('car'))

In [ ]:
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = requests.get(url).content
    image = np.asarray(bytearray(resp), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    # return the image
    return image

In [ ]:
def getVehicleCount(retrievalTime):
    url_traffic = 'https://api.data.gov.sg/v1/transport/traffic-images'

    params = {"date_time": retrievalTime.strftime("%Y-%m-%dT%H:%M:%S")}

    print(params)

    data = json.loads(requests.get(url_traffic, params=params).content)

    # Get image URL for the requested location
    try:
        url = [d.get('image', None) for d in data['items'][0]['cameras'] if d.get('location', None) == {'latitude': 1.357098686, 'longitude': 103.902042}][0]
    except:
        print("Unable to get data for {0}".format(retrievalTime))
        return -1 # Invalid Data

    image = url_to_image(url)
    cv2plt(image)

    [fboxes, fclasses] = yoloV3Detect(image)
    pltDetect(image, fboxes, fclasses, classes)

    return (len(fboxes))

In [ ]:
def getVehicleCountFile(imageFile):
    image = cv2.imread(imageFile)
    # cv2plt(image)

    [fboxes, fclasses] = yoloV3Detect(image)
    pltDetect(image, fboxes, fclasses, classes)

    return (len(fboxes))

**Nearest Weather Station will be 36 Kim Chuan Road**
Device ID: S43

In [ ]:
def getRainfall(retrievalTime):
    url_weather = 'https://api.data.gov.sg/v1/environment/rainfall'
    params = {"date_time": retrievalTime.strftime("%Y-%m-%dT%H:%M:%S")} 

    data_weather = json.loads(requests.get(url_weather, params=params).content)

    # Get rainfall for the requested location
    try:
        rainfall = [d.get('value', None) for d in data_weather.get('items')[0].get('readings') if d.get('station_id', None) == "S43"][0]
    except:
        print("Unable to get data for {0}".format(retrievalTime))
        return -1 # Invalid Data

    return rainfall

In [ ]:
# This method will run inference on images already pulled from DataGov

# Declare Empty DataFrame
train_df = pd.DataFrame(columns=['Datetime', 'Vehicle Count'])

for root, dirs, files in os.walk('images'):
    files.sort()
    for filename in files[8618:]:
        imgFile = os.path.join(root, filename)
        print(filename)
        vehicleCount = getVehicleCountFile(imgFile)
        a_series = pd.Series([filename, vehicleCount], index = train_df.columns)
        train_df = train_df.append(a_series, ignore_index=True)
        train_df['Datetime'] = train_df['Datetime'].str.replace(".jpg", "").str.replace("T", " ")
        #train_df['Datetime'] = pd.to_datetime(train_df.Datetime,format='%Y-%m-d %H-%M-%S')
        train_df.to_excel("DataGov.xlsx", index=False)

In [ ]:
# This method will take long time. As it run inference as it is pulling image data from DataGov

# Declare Empty DataFrame
train_df = pd.DataFrame(columns=['Datetime', 'Vehicle Count', 'Rainfall'])

start = datetime.now().replace(month=1, day=1, hour=0, minute=0, second=0)

for dy in range(365, -1, -1):
    whichday = start - timedelta(days=dy)
    for hr in range(24):
        print("Iteration {} {}".format(dy, hr))
        now = whichday + timedelta(hours=hr)
        vehicleCount = getVehicleCount(now)
        rainfall = getRainfall(now)
        print("Vehicle Count is {0}.".format(vehicleCount))
        print("Rainfall is {0} mm".format(rainfall))
        a_series = pd.Series([now, vehicleCount, rainfall], index = train_df.columns)
        train_df = train_df.append(a_series, ignore_index=True)
        train_df['Datetime'] = pd.to_datetime(train_df.Datetime,format='%d-%m-%Y %H:%M') 
    train_df.to_excel(".\DataGov.xlsx", index=False)

### To pull latest past 3 hour data

In [1]:
# This method will take long time. As it run inference as it is pulling image data from DataGov
def getLastData(retrievalTime):
    train_df = pd.DataFrame(columns=['Datetime', 'Rainfall', 'Vehicle Count'])

    for hr in range(3, 0, -1):
        print("Iteration {}".format(hr))
        now = retrievalTime - timedelta(hours=hr)
        rainfall = getRainfall(now)
        vehicleCount = getVehicleCount(now)
        print("Rainfall is {0} mm".format(rainfall))
        print("Vehicle Count is {0}.".format(vehicleCount))
        now = now.strftime("%Y-%m-%dT%H:%M:%S")
        a_series = pd.Series([now, rainfall, vehicleCount], index = train_df.columns)
        train_df = train_df.append(a_series, ignore_index=True)

    return train_df

# We are using **Prophet** to do Time Series Prediction

In [6]:
train = pd.read_excel('train.xlsx')
print(train.head())
print("***")
print(train.tail())
print("***")
print(train.info())

In [ ]:
sgp = pytz.timezone('Asia/Singapore')
d = datetime.datetime.now(sgp)
train = pd.concat([train, getLastData(d)])
print(train.tail())
train.reset_index(drop=True)

In [7]:
plt.figure(figsize=(10, 10))
plt.plot_date(train['Datetime'], train['Vehicle Count'], fmt="b-")
plt.grid(True)
plt.show()

In [8]:
train.columns = ['ds', 'rainfall', 'y']
train['ds']= pd.to_datetime(train['ds'])
train.head()

In [9]:
m = Prophet(changepoint_prior_scale=0.01)
m.add_country_holidays(country_name='SG')
m.add_regressor('rainfall')
m.fit(train)

In [27]:
future = m.make_future_dataframe(periods=3, freq='H')
future['rainfall'] = train['rainfall']
future.loc[-3:, 'rainfall'] = train.iloc[-1]['rainfall']
future.info()

In [29]:
fcst = m.predict(future)
fig = m.plot(fcst)

In [38]:
fcst['yhat'] = fcst['yhat'].clip(lower=0).round()
fcst['yhat'] = fcst['yhat'].astype(int)
print(fcst.describe())
print("**")
print(fcst.info())

In [67]:
fig = m.plot_components(fcst)

## Observation
1. Number of cars using this expressway was trending down from 2021 to 2022
2. Highest usage is on Saturday 
3. Peak hour usually occurs about close to 9am and close to 5pm, which can be explained to start and end of working hours

# Generate Prediction for next 3 Hours

In [66]:
prediction = fcst.tail(3)['yhat'].reset_index()

print("Prediction of number of cars for next 3 hours: \r\nHour 1: {0} \r\nHour 2: {1} \r\nHour 3: {2}".format(prediction.iloc[0].yhat, prediction.iloc[1].yhat,prediction.iloc[2].yhat))

# Enhancements to be made:

1. Include pre-Holiday effect (e.g. Christmas eve, New Year Eve etc.): include columns lower_window and upper_window which extend the holiday out 
2. Get weather prediction (convert descriptive text from Data.Gov to quantitative rainfall)
3. Saving of Prophet model